# import

In [1]:
import numpy as np
import pandas as pd
import glob
import sqlite3
import datetime
from my_library.horse import *

# funcs

In [ ]:
def load_csv(load_path):
    df = pd.read_csv(load_path, index_col=0)
    return df

# classes

In [ ]:
class DataFramePreProcessing():

    
    def __init__(self, path_, is_daw=False):
        self.path_ = path_
        self.is_daw = is_daw

        
    def load_df(self):
        if self.is_daw:
            d='d'
        else:
            d=''
        FILE = glob.glob(self.path_)
        df = pd.read_csv(FILE[0])
        df = df.rename(columns={df.columns[0]:'nan',df.columns[1]:'nan',df.columns[2]:'nan',\
                                    df.columns[3]:'day',df.columns[4]:'nan',df.columns[5]:d+'open',\
                                    df.columns[6]:d+'high',df.columns[7]:d+'low',df.columns[8]:d+'close',\
                                       df.columns[9]:d+'volume',})
        df = df.drop('nan',axis=1)
        df = df.drop(df.index[0])
        df['day'] = pd.to_datetime(df['day'],format='%Y/%m/%d')
        df.set_index('day',inplace=True)

        return df.astype(float)
    
class DBInterface():
    
    
    def __init__(self):
        self.db = None
        self.is_initialized = False
        
        
    def make_db(self,path_):
        if not self.is_initialized:
            db = sqlite3.connect(
                    path_,              #ファイル名
                    isolation_level=None,
                    )
            self.db = db
        else:
            print("already initialized")
        
    
    def load_db(self, path_):
        if not self.is_initialized:
            db = sqlite3.connect(
                    path_,              #ファイル名
                    isolation_level=None,
                    )
            self.db = db
        else:
            print("already initialized")
        
        
    def show_tables(self):
        sql = """SELECT NAME FROM sqlite_master WHERE TYPE='table'; """
        for i in self.db.execute(sql):
            print(i[0])
            
            
    def create_table(self,sql):
        self.db.execute(sql)
        
        
    def drop_table(self, table_name):
        self.db.execute("""DROP TABLE IF EXISTS %s""" % table_name)
        
    
    def show_column(self, table):
        self.db.execute("""SELECT * FROM %s""" % table)
        names = list(map(lambda x: x[0], self.db.cursor().description))
        print(names)
        
        
    def select_show(self, attribute, table):
        for item in self.db.execute("""SELECT id FROM Meigara WHERE name = %s""" % table):
            print(item)
            
            
    def return_db(self):
        return self.db
    
    
    def make_Results_tuple(self, df, id_):
        Hiashis = pd.DataFrame()
        Hiashis['id'] = [id_] * len(df)
        Hiashis['day'] = df.reset_index()['day'].astype(str)
        Hiashis['open'] = df['open'].tolist()
        Hiashis['high'] = df['high'].tolist()
        Hiashis['low'] = df['low'].tolist()
        Hiashis['close'] = df['close'].tolist()
        Hiashis['volume'] = df['volume'].tolist()
        
        return list(Hiashis.itertuples(index=False, name=None)) 
    
    
    def execute_many_Hiashis(self, df, name):
        id_ = self.return_id(name)
        try:
            Hiashis = self.make_Hiashis_tuple(df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except :
            print("UNIQUE constraint failed: Hiashis.id, Hiashis.date")

        
    def return_id(self, name):
        sql = """SELECT id FROM Meigaras WHERE name='%s' """ % name
        c = self.db.cursor()
        id_ = c.execute(sql).fetchone()[0]
        return id_
    
    
    def update_Hiashi(self,df,name):
        try:
            id_ = self.return_id(name)
        except:
            print("No such Meigaras.name !")
        
        sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = %s""" %id_
        table_date = self.db.execute(sql).fetchone()[0] 
        
        try:
            new_df = df[df.index>table_date]
            Hiashis = self.make_Hiashis_tuple(new_df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except:
            print("Insert failed.")
            
    
    
    def make_Ayumines_tuple(self, df,id_):
######### date どないすんねん
        date_ = None
        Ayumines = pd.DataFrame()
        Ayumines['id'] = [id_] * len(df)
        Ayumines['date'] = [date_] * len(df)#df.reset_index()['時刻'].astype(str)
        Ayumines['time'] = df['時刻'].tolist()
        Ayumines['price'] = df['値段'].tolist()
        Ayumines['volume'] = df['株数'].tolist()
        Ayumines['total_price'] = df['金額'].tolist()

        return list(Ayumines.itertuples(index=False, name=None)) 
    
    
    def execute_many_Ayumines(self, df, name):
        id_ = self.return_id(name)
        try:
            Ayumines = self.make_Ayumines_tuple(df,id_)
            self.db.executemany( "INSERT INTO Ayumines VALUES (?,?,?,?,?,?)", Ayumines )
        except :
            print("UNIQUE constraint failed: Hiashis.id, Hiashis.date")
            
        
    def update_Ayumine(self,df,name):
        try:
            id_ = self.return_id(name)
        except:
            print("No such Meigaras.name !")
        
        sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = %s""" %id_
        table_date = self.db.execute(sql).fetchone()[0] 
        
        try:
            new_df = df[df.index>table_date]
            Hiashis = self.make_Hiashis_tuple(new_df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except:
            print("Insert failed.")

# 一旦, クラス使わないで, dbいじる

In [ ]:
sql = """
CREATE TABLE Test(
id INTEGER PRIMARY KEY AUTOINCREMENT,
name VARCHAR(40)
);"""

In [ ]:
sql = """
DROP TABLE Horse_Results;"""
db.execute(sql)

In [ ]:
db.execute(sql)

In [ ]:
sql = """
INSERT  INTO Test(name) VALUES('hoge_hoge');
"""

In [ ]:
db.execute(sql)

In [ ]:
db = sqlite3.connect(
    "horse.db",              #ファイル名
    isolation_level=None,
)

In [ ]:
sql = """SELECT NAME FROM sqlite_master WHERE TYPE='table'; """
for i in db.execute(sql):
    print(i[0])
    

In [ ]:
horse_results.index.tolist()

In [ ]:
horse_results.columns

In [ ]:
sql = """SELECT * FROM Horse_Results;"""
db.execute(sql)

In [ ]:
sql = """CREATE TABLE Horse_Results(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        horse_id INTEGER,
        date VARCHAR(40),
        kaisai VARCHAR(40),
        tenki VARCHAR(40),
        round INTEGER,
        race_name VARCHAR(40),
        tousu INTEGER,
        wakuban INTEGER,
        umaban INTEGER,
        odds REAL,
        ninki REAL,
        chakujun VARCHAR(40),
        kishu VARCHAR(40),
        kinryo  REAL,
        kyori  VARCHAR(40),
        baba  VARCHAR(20),
        time VARCHAR(20),
        chakusa REAL,
        tsu_ka VARCHAR(20),
        pace VARCHAR(20),
        nobori REAL,
        taiju VARCHAR(20),
        kachiuma VARCHAR(40),
        shokin REAL
    );
"""


# INSERT UMA DB

In [ ]:
db.execute(sql)

In [ ]:
sql = """

INSERT  INTO Test (name) VALUES ('hoge_hoge')
;
"""

In [ ]:
horse_results['天気']

In [ ]:
horse_results.columns

In [ ]:

Horse_Results = pd.DataFrame()
Horse_Results['horse_id'] = horse_results.index.tolist()
Horse_Results['date'] = horse_results['日付'].tolist()#df.reset_index()['時刻'].astype(str)
Horse_Results['kaisai'] = horse_results['開催'].tolist()
Horse_Results['tenki'] = horse_results['天気'].tolist()
Horse_Results['round'] = horse_results['R'].tolist()
Horse_Results['race_name'] = horse_results['レース名'].tolist()
Horse_Results['tousu'] = horse_results['頭数'].tolist()#horse_results.reset_index()['時刻'].astype(str)
Horse_Results['wakuban'] = horse_results['枠番'].tolist()
Horse_Results['umaban'] = horse_results['馬番'].tolist()
Horse_Results['odds'] = horse_results['オッズ'].tolist()
Horse_Results['ninki'] = horse_results['人気'].tolist()
Horse_Results['chakujun'] = horse_results['着順'].tolist()
Horse_Results['kishu'] = horse_results['騎手'].tolist()
Horse_Results['kinryo'] = horse_results['斤量'].tolist()
Horse_Results['kyori'] = horse_results['距離'].tolist()
Horse_Results['baba'] = horse_results['馬場'].tolist()
Horse_Results['time'] = horse_results['タイム'].tolist()
Horse_Results['chakusa'] = horse_results['着差'].tolist()
Horse_Results['tsu_ka'] = horse_results['通過'].tolist()
Horse_Results['pace'] = horse_results['ペース'].tolist()
Horse_Results['nobori'] = horse_results['上り'].tolist()
Horse_Results['taiju'] = horse_results['馬体重'].tolist()
Horse_Results['kachiuma'] = horse_results['勝ち馬(2着馬)'].tolist()
Horse_Results['shokin'] = horse_results['賞金'].tolist()

Horse_Results_tuple =  list(Horse_Results.itertuples(index=False, name=None)) 






In [ ]:
Horse_Results_tuple

In [ ]:

db.executemany( """
INSERT INTO Horse_Results (horse_id,date,kaisai,tenki,round,race_name,tousu,wakuban,umaban,odds,ninki,chakujun,kishu,kinryo,kyori,baba,time,chakusa,tsu_ka,pace,nobori,taiju,kachiuma,shokin) 
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
, Horse_Results_tuple)

In [ ]:
for data in horse_results:
    print(data)
    break

In [ ]:
horse_results

In [ ]:
db.execute(sql)

In [ ]:
sql = """
DROP TABLE Test;
"""

In [ ]:
db.execute(sql)

# path_

# Table

horse_table \
jocky_table \
race_table \
trainer_table \
owner_table \
breeder_table \


In [ ]:
save_path = '/Users/rince/Desktop/Horse/Data/saishin/return.csv'
return_tables = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/results.csv'
results = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/peds.csv'
peds = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/horse_results.csv'
horse_results = load_csv(save_path)

In [ ]:
horse_results.columns

In [ ]:
horse_results.index

In [ ]:
return_tables.head()

In [ ]:
# peds.index.name = 'horse_id'
peds.head()

In [ ]:
# horse_results.index.name = 'horse_id'
horse_results.head()

In [ ]:
horse_results.head()

In [ ]:
cursor.execute_many(df_nissan_new,'02Nissan')


In [ ]:
db = cursor.return_db()
sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = 2"""
count = 0
for i in  db.execute(sql):
    date_ = i[0]
    count+=1
    if count>=10:
        break

In [ ]:
db.execute(sql).fetchone()[0]

In [ ]:
date_

# ユニークindex

In [ ]:
sql = """CREATE UNIQUE INDEX date_check
 ON Hiashis(meigara_id,date);"""
cursor.return_db().execute(sql)

# Results table

In [ ]:
sql = """
    CREATE TABLE Results (
        id INTEGER ,
        chakujun VARCHAR(40),
        wakuban INTEGER,
        umaban INTEGER,
        umamei VARCHAR(40),
        seirei VARCHAR(40),
        kinryo REAL,
        kishu VARCHAR(40),
        time VARCHAR(40),
        chakusa VARCHAR(40),
        tansho VARCHAR(40),
        ninki REAL,
        umataiju VARCHAR(40),
        chokyoshi VARCHAR(40),
        course_len INTEGER,
        weather VARCHAR(40),
        race_type VARCHAR(40),
        ground_state VARCHAR(40),
        date VARCHAR(40),
        horse_id INTEGER,
        jocky_id INTEGER 
    );
"""
cursor.create_table(sql)

# Return table

In [ ]:
sql = """
    CREATE TABLE Hiashis (
        meigara_id INTEGER,
        date VARCHAR(20),
        open REAL,
        high REAL,
        low  REAL,
        close REAL,
        volume REAL
    );
"""
cursor.create_table(sql)

# Peds table

In [ ]:
sql = """
    CREATE TABLE Ayumines (
        id INTEGER ,
        date VARCHAR(20),
        time VARCHAR(20),
        price REAL,
        volume REAL,
        total_price  REAL
    );
"""
cursor.create_table(sql)

# Horse_results table

In [ ]:
sql = """
    CREATE TABLE Ayumines (
        id INTEGER ,
        date VARCHAR(20),
        time VARCHAR(20),
        price REAL,
        volume REAL,
        total_price  REAL
    );
"""
cursor.create_table(sql)

# SQLAlchemy

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import os

In [3]:
database_file = os.path.join(os.path.abspath(os.getcwd()),'data.db')

おまじない

In [4]:
enigine = create_engine('sqlite:///'+database_file,convert_unicode=True, echo=True)
#  pool_pre_ping=True

In [5]:
db_session = scoped_session(
    sessionmaker(
        autocommit=False,
        autoflush=False,
        bind = enigine
    )
)

In [6]:
Base = declarative_base()
Base.query = db_session.query_property()

In [7]:
from sqlalchemy import Column, Integer, Float, String, desc

# horse_results table

In [24]:
# テーブル記述
class Horse(Base):
    __tablename__= "horse_results"
    # 主キーないとエラー
    # 日付 : date と horse_id で一意に区別できる
    id = Column(Integer,primary_key=True)
    horse_id = Column(Integer,unique=False)
    date = Column(String(40),unique=False)
    place = Column(String(40),unique=False)
    weather = Column(String(10),unique=False)
    round_ = Column(Integer,unique=False)
    race_name = Column(String(40),unique=False)
    # 映像全部null
    eizo = Column(String(10),unique=False)
    # 頭数
    horse_len = Column(Integer,unique=False)
    wakuban = Column(Integer,unique=False)
    umaban = Column(Integer,unique=False)
    odds = Column(Float,unique=False)
    ninki = Column(Integer,unique=False)
    chakujun = Column(Integer,unique=False)
    jockey = Column(String(20),unique=False)
    kinryo = Column(Float,unique=False)
    kyori = Column(String(20),unique=False)
    baba = Column(String(10),unique=False)
    baba_index = Column(String(10),unique=False)
    time = Column(String(20),unique=False)
    chakusa = Column(Float,unique=False)
    time_index = Column(String(10),unique=False)
    # 通過
    tsuuka = Column(String(20),unique=False)
    pace = Column(String,unique=False)
    nobori = Column(Float,unique=False)
    weight = Column(String(20),unique=False)
    comment = Column(String,unique=False)
    bikou = Column(String(10),unique=False)
    kachiuma = Column(String(10),unique=False)
    prize  = Column(Float,unique=False)



    def __init__(self,horse_id, date, place, weather, round_, race_name, eizo, horse_len, wakuban, umaban, odds,
    ninki, chakujun, jockey, kinryo, kyori, baba, baba_index, time, chakusa, time_index, tsuuka, pace, nobori,
    weight, comment, bikou, kachiuma, prize):
        
        self.horse_id = horse_id
        self.date = date
        self.place = place
        self.weather = weather
        self.round_ = round_
        self.race_name = race_name
        # 映像全部null
        self.eizo = eizo
        # 頭数
        self.horse_len = horse_len
        self.wakuban = wakuban
        self.umaban = umaban
        self.odds = odds
        self.ninki = ninki
        self.chakujun = chakujun
        self.jockey = jockey
        self.kinryo = kinryo
        self.kyori = kyori
        self.baba = baba
        self.baba_index = baba_index
        self.time = time
        self.chakusa = chakusa
        self.time_index = time_index
        # 通過
        self.tsuuka = tsuuka
        self.pace = pace
        self.nobori = nobori
        self.weight = weight
        self.comment = comment
        self.bikou = bikou
        self.kachiuma = kachiuma
        self.prize  = prize





# peds table

In [9]:
# テーブル記述
class Peds(Base):
    __tablename__= "peds"
    id = Column(Integer,primary_key=True)
    horse_id = Column(Integer,unique=True)
    peds_0 = Column(String(40),unique=False)
    peds_1 = Column(String(40),unique=False)
    peds_2 = Column(String(40),unique=False)
    peds_3 = Column(String(40),unique=False)
    peds_4 = Column(String(40),unique=False)
    peds_5 = Column(String(40),unique=False)
    peds_6 = Column(String(40),unique=False)
    peds_7 = Column(String(40),unique=False)
    peds_8 = Column(String(40),unique=False)
    peds_9 = Column(String(40),unique=False)
    peds_10 = Column(String(40),unique=False)

    peds_11 = Column(String(40),unique=False)
    peds_12 = Column(String(40),unique=False)
    peds_13 = Column(String(40),unique=False)
    peds_14 = Column(String(40),unique=False)
    peds_15 = Column(String(40),unique=False)
    peds_16 = Column(String(40),unique=False)
    peds_17 = Column(String(40),unique=False)
    peds_18 = Column(String(40),unique=False)
    peds_19 = Column(String(40),unique=False)
    peds_20 = Column(String(40),unique=False)

    peds_21 = Column(String(40),unique=False)
    peds_22 = Column(String(40),unique=False)
    peds_23 = Column(String(40),unique=False)
    peds_24 = Column(String(40),unique=False)
    peds_25 = Column(String(40),unique=False)
    peds_26 = Column(String(40),unique=False)
    peds_27 = Column(String(40),unique=False)
    peds_28 = Column(String(40),unique=False)
    peds_29 = Column(String(40),unique=False)
    peds_30 = Column(String(40),unique=False)

    peds_31 = Column(String(40),unique=False)
    peds_32 = Column(String(40),unique=False)
    peds_33 = Column(String(40),unique=False)
    peds_34 = Column(String(40),unique=False)
    peds_35 = Column(String(40),unique=False)
    peds_36 = Column(String(40),unique=False)
    peds_37 = Column(String(40),unique=False)
    peds_38 = Column(String(40),unique=False)
    peds_39 = Column(String(40),unique=False)
    peds_40 = Column(String(40),unique=False)

    peds_41 = Column(String(40),unique=False)
    peds_42 = Column(String(40),unique=False)
    peds_43 = Column(String(40),unique=False)
    peds_44 = Column(String(40),unique=False)
    peds_45 = Column(String(40),unique=False)
    peds_46 = Column(String(40),unique=False)
    peds_47 = Column(String(40),unique=False)
    peds_48 = Column(String(40),unique=False)
    peds_49 = Column(String(40),unique=False)
    peds_50 = Column(String(40),unique=False)

    peds_51 = Column(String(40),unique=False)
    peds_52 = Column(String(40),unique=False)
    peds_53 = Column(String(40),unique=False)
    peds_54 = Column(String(40),unique=False)
    peds_55 = Column(String(40),unique=False)
    peds_56 = Column(String(40),unique=False)
    peds_57 = Column(String(40),unique=False)
    peds_58 = Column(String(40),unique=False)
    peds_59 = Column(String(40),unique=False)
    peds_60 = Column(String(40),unique=False)
    peds_61 = Column(String(40),unique=False)

    
    def __init__(self,horse_id,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,peds_11,peds_12,peds_13,peds_14,peds_15,peds_16,peds_17,peds_18,peds_19,
                    peds_20,peds_21,peds_22,peds_23,peds_24,peds_25,peds_26,peds_27,peds_28,peds_29,peds_30,peds_31,peds_32,peds_33,peds_34,peds_35,peds_36,peds_37,peds_38,peds_39,
                    peds_40,peds_41,peds_42,peds_43,peds_44,peds_45,peds_46,peds_47,peds_48,peds_49,peds_50,peds_51,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,
                    peds_60,peds_61):
        
        self.horse_id = horse_id

        self.peds_0 = peds_0
        self.peds_1 = peds_1
        self.peds_2 = peds_2
        self.peds_3 = peds_3
        self.peds_4 = peds_4
        self.peds_5 = peds_5
        self.peds_6 = peds_6
        self.peds_7 = peds_7
        self.peds_8 = peds_8
        self.peds_9 = peds_9
        self.peds_10 = peds_10
        self.peds_11 = peds_11
        self.peds_12 = peds_12
        self.peds_13 = peds_13
        self.peds_14 = peds_14
        self.peds_15 = peds_15
        self.peds_16 = peds_16
        self.peds_17 = peds_17
        self.peds_18 = peds_18
        self.peds_19 = peds_19
        self.peds_20 = peds_20
        self.peds_21 = peds_21
        self.peds_22 = peds_22
        self.peds_23 = peds_23
        self.peds_24 = peds_24
        self.peds_25 = peds_25
        self.peds_26 = peds_26
        self.peds_27 = peds_27
        self.peds_28 = peds_28
        self.peds_29 = peds_29
        self.peds_30 = peds_30
        self.peds_31 = peds_31
        self.peds_32 = peds_32
        self.peds_33 = peds_33
        self.peds_34 = peds_34
        self.peds_35 = peds_35
        self.peds_36 = peds_36
        self.peds_37 = peds_37
        self.peds_38 = peds_38
        self.peds_39 = peds_39
        self.peds_40 = peds_40
        self.peds_41 = peds_41
        self.peds_42 = peds_42
        self.peds_43 = peds_43
        self.peds_44 = peds_44
        self.peds_45 = peds_45
        self.peds_46 = peds_46
        self.peds_47 = peds_47
        self.peds_48 = peds_48
        self.peds_49 = peds_49
        self.peds_50 = peds_50
        self.peds_51 = peds_51
        self.peds_52 = peds_52
        self.peds_53 = peds_53
        self.peds_54 = peds_54
        self.peds_55 = peds_55
        self.peds_56 = peds_56
        self.peds_57 = peds_57
        self.peds_58 = peds_58
        self.peds_59 = peds_59
        self.peds_60 = peds_60
        self.peds_61 = peds_61

        




# Results table

In [10]:
class Results(Base):
    __tablename__= "results"
    # 主キーないとエラー
    id = Column(Integer,primary_key=True)
    race_id = Column(Integer,unique=False)
    chakujun = Column(Integer,unique=False)
    wakuban = Column(Integer,unique=False)
    umaban = Column(Integer,unique=False)
    umamei = Column(String(40),unique=False)
    seirei = Column(String(40),unique=False)
    kinryo = Column(Float,unique=False)
    # 騎手
    jockey = Column(String(20),unique=False)
    time = Column(String(20),unique=False)
    chakusa = Column(String(20),unique=False)
    # 単勝
    tansho_odds = Column(Float,unique=False)
    # 人気
    ninki = Column(Integer,unique=False)
    # 馬体重
    weight = Column(String(10),unique=False)
    chokyosi = Column(String(20),unique=False)
    course_len = Column(Integer,unique=False)
    weather = Column(String(10),unique=False)
    race_type = Column(String(10),unique=False)
    ground_state = Column(String(10),unique=False)
    date = Column(String(20),unique=False)
    horse_id = Column(Integer,unique=False)
    jockey_id = Column(Integer,unique=False)


    def __init__(self, race_id, chakujun, wakuban, umaban, umamei, seirei, kinryo, jockey, time, chakusa, tansho_odds, ninki, weight, chokyosi, course_len, weather, race_type, ground_state, date, horse_id, jockey_id):
        self.race_id = race_id
        self.chakujun = chakujun
        self.wakuban = wakuban
        self.umaban = umaban
        self.umamei = umamei
        self.seirei = seirei
        self.kinryo = kinryo
        self.jockey = jockey
        self.time = time
        self.chakusa = chakusa
        self.tabsho_odds = tansho_odds
        self.ninki = ninki
        self.weight = weight
        self.chokyosi = chokyosi
        self.course_len = course_len
        self.weather = weather
        self.race_type = race_type
        self.ground_state = ground_state
        self.date = date 
        self.horse_id = horse_id
        self.jockey_id = jockey_id

# Return table

In [11]:
class Return(Base):
    __tablename__= "return"
    # 主キーないとエラー
    # 日付 : date と horse_id で一意に区別できる
    id = Column(Integer,primary_key=True)
    race_id = Column(Integer,unique=False)
    kaime = Column(String(10),unique=False)
    atari = Column(String(20),unique=False)
    haraimodoshi = Column(String(20),unique=False)
    ninki = Column(String(10),unique=False)


    def __init__(self,race_id, kaime, atari, haraimodoshi, ninki):
        self.race_id = race_id
        self.kaime = kaime
        self.atari = atari
        self.haraimodoshi = haraimodoshi
        self.ninki = ninki


In [ ]:

# Base.metadata.create_all(bind=enigine)

# race_id_list

In [12]:

race_id_list = ['2022050201{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050202{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022090209{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022090210{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022030103{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022030104{}'.format(str(i).zfill(2)) for i in range(1,13)]

####
race_id_list += ['2022050203{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050204{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022090211{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022090212{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022030105{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022030106{}'.format(str(i).zfill(2)) for i in range(1,13)]




In [13]:
def add_horse_results(horse_results):
    for index, attr in horse_results.iterrows():
        # print(attr.values)
        varlis = attr.values
        row =  Horse(
                    index,
                    varlis[0],
                    varlis[1],
                    varlis[2],
                    varlis[3],
                    varlis[4],
                    varlis[5],
                    varlis[6],
                    varlis[7],
                    varlis[8],
                    varlis[9],
                    varlis[10],
                    varlis[11],
                    varlis[12],
                    varlis[13],
                    varlis[14],
                    varlis[15],
                    varlis[16],
                    varlis[17],
                    varlis[18],
                    varlis[19],
                    varlis[20],
                    varlis[21],
                    varlis[22],
                    varlis[23],
                    varlis[24],
                    varlis[25],
                    varlis[26],
                    varlis[27]
                    )
        
        tmp_horse_id = Horse.horse_id
        tmp_date = Horse.date
        db = db_session.query(Horse).all()
        is_dupulicated = False
        for row in db:
            if row.horse_id==tmp_horse_id and row.date==tmp_date:
                is_dupulicated = True
                break
        if is_dupulicated:
            continue
        db_session.add(row)
    db_session.commit()


In [14]:
def add_peds(peds):
    for index, attr in peds.iterrows():
        varlis = attr.values
        row = Peds(
            index,
            varlis[0],
            varlis[1],
            varlis[2],
            varlis[3],
            varlis[4],
            varlis[5],
            varlis[6],
            varlis[7],
            varlis[8],
            varlis[9],
            varlis[10],
            varlis[11],
            varlis[12],
            varlis[13],
            varlis[14],
            varlis[15],
            varlis[16],
            varlis[17],
            varlis[18],
            varlis[19],
            varlis[20],
            varlis[21],
            varlis[22],
            varlis[23],
            varlis[24],
            varlis[25],
            varlis[26],
            varlis[27],
            varlis[28],
            varlis[29],
            varlis[30],
            varlis[31],
            varlis[32],
            varlis[33],
            varlis[34],
            varlis[35],
            varlis[36],
            varlis[37],
            varlis[38],
            varlis[39],
            varlis[40],
            varlis[41],
            varlis[42],
            varlis[43],
            varlis[44],
            varlis[45],
            varlis[46],
            varlis[47],
            varlis[48],
            varlis[49],
            varlis[50],
            varlis[51],
            varlis[52],
            varlis[53],
            varlis[54],
            varlis[55],
            varlis[56],
            varlis[57],
            varlis[58],
            varlis[59],
            varlis[60],
            varlis[61]
        )
        # 重複チェック
        tmp_horse_id = Peds.horse_id
        db = db_session.query(Peds).all()
        is_dupulicated = False
        for row in db:
            if row.horse_id==tmp_horse_id:
                is_dupulicated = True
                break
        if is_dupulicated:
            continue
        db_session.add(row)
        
    db_session.commit()
        

In [15]:
def add_results(results):
    
    for index, attr in results.iterrows():
        varlis = attr.values
        row = Results(
            index,
            varlis[0],
            varlis[1],
            varlis[2],
            varlis[3],
            varlis[4],
            varlis[5],
            varlis[6],
            varlis[7],
            varlis[8],
            varlis[9],
            varlis[10],
            varlis[11],
            varlis[12],
            varlis[13],
            varlis[14],
            varlis[15],
            varlis[16],
            varlis[17],
            varlis[18],
            varlis[19]
        )
        is_dupulicated = False
        db = db_session.query(Results).all()
        tmp_race_id = Results.race_id
        tmp_horse_id = Results.horse_id
        # race_id と horse_id で一意にデータの区別がつく
        for row in db:
            if row.race_id==tmp_race_id and row.horse_id==tmp_horse_id:
                is_dupulicated = True
                break
        if is_dupulicated:
            continue
        db_session.add(row)
    db_session.commit()
        

In [16]:
def add_return(return_tables):
    for index, attr in return_tables.iterrows():
        varlis = attr.values
        row = Return(
            index,
            varlis[0],
            varlis[1],
            varlis[2],
            varlis[3]
        )
        is_dupulicated = False
        db = db_session.query(Return).all()
        tmp_race_id = Return.race_id
        tmp_kaime = Return.kaime
        # race_id と horse_id で一意にデータの区別がつく
        for row in db:
            if row.race_id==tmp_race_id and row.kaime==tmp_kaime:
                is_dupulicated = True
                break
        if is_dupulicated:
            continue
        db_session.add(row)
    db_session.commit()

# データ追加

In [17]:
results = results_scrape(race_id_list)
horse_id_list = results['horse_id'].astype(str).unique()
horse_results = horse_results_scrape(horse_id_list)
peds = peds_scrape(horse_id_list)
returns = return_scrape(race_id_list)

In [22]:
peds

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2019103122,リオンディーズ 2013 黒鹿毛 [血統][産駒] Mr. Prospector系,ショウナンガッド 2003 鹿毛 [血統][産駒] FNo.[11-d],キングカメハメハ 2001 鹿毛 [血統][産駒],シーザリオ 2002 青毛 [血統][産駒],フジキセキ 1992 青鹿毛 [血統][産駒],シャンラン Xianlang(愛) 1993 鹿毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],スペシャルウィーク 1995 黒鹿毛 [血統][産駒],キロフプリミエール Kirov Premiere(英) 1990 鹿毛 [血統][産駒],...,In Reality,Millicent,Northern Dancer,Special,Lorenzaccio,Vive La Reine,Queen's Hussar,La Paiva,Blakeney,Death Ray
2019106491,ラニ 2013 芦毛 [血統][産駒] Seattle Slew系,レーヴドベティ 2008 鹿毛 [血統][産駒] FNo.[4-i],Tapit (米) 2001 芦毛 [血統][産駒],ヘヴンリーロマンス 2000 鹿毛 [血統][産駒],ロックオブジブラルタル 1999 鹿毛 [血統][産駒],レーブドグランパ 1994 鹿毛 [血統][産駒],Pulpit 1994 鹿毛 [血統][産駒],Tap Your Heels 1996 芦毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ファーストアクト First Act(愛) 1986 鹿毛 [血統][産駒],...,Bold Lad,River Lady,Raise a Native,Gold Digger,Halo,Raise the Standard,The Minstrel,Suprina,Brigadier Gerard,Marike
2019106678,ドレフォン Drefong(米) 2013 鹿毛 [血統][産駒] Storm Bird系,ホーマンピクシー 2007 栗毛 [血統][産駒] FNo.[4],Gio Ponti (米) 2005 鹿毛 [血統][産駒],Eltimaas 2007 鹿毛 [血統][産駒],キングカメハメハ 2001 鹿毛 [血統][産駒],タイトーク Tie Talk(米) 1994 芦毛 [血統][産駒],Tale of the Cat (米) 1994 黒鹿毛 [血統][産駒],Chipeta Springs 1989 鹿毛 [血統][産駒],Ghostzapper 2000 鹿毛 [血統][産駒],Najecam 1993 栗毛 [血統][産駒],...,Blakeney,The Dancer,Mr. Prospector,Hopespringseternal,Al Hattab,Desperate Action,Raise a Native,Old Goat,Elocutionist,Rolltosweets
2019109076,ブラックタイド 2001 黒鹿毛 [血統][産駒] Halo系,タイホウパーリオ 2008 鹿毛 [血統][産駒] FNo.[A13],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],キングカメハメハ 2001 鹿毛 [血統][産駒],ローレルアンジュ 1999 鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],Alzao 1980 鹿毛 [血統][産駒],Burghclere 1977 鹿毛 [血統][産駒],...,Blakeney,The Dancer,Riverman,Irish Star,Northfields,ツリーオブノレッジ,Raise a Native,Sweet Tooth,Quibu,Art Teacher
2019105583,コパノリッキー 2010 栗毛 [血統][産駒] Halo系,フィールザタレント 2007 鹿毛 [血統][産駒] FNo.[7],ゴールドアリュール 1999 栗毛 [血統][産駒],コパノニキータ 2001 栗毛 [血統][産駒],Theatrical 1982 黒鹿毛 [血統][産駒],Feel the Capote 1993 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ニキーヤ Nikiya(米) 1993 鹿毛 [血統][産駒],ティンバーカントリー Timber Country(米) 1992 栗毛 [血統][産駒],ニホンピロローズ 1996 栗毛 [血統][産駒],...,Hail to Reason,Pange,Bold Reasoning,My Charmer,Bald Eagle,Hidden Talent,Marshua's Dancer,Tudor Twist,Tulyar,Up Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017103104,グランプリボス 2008 鹿毛 [血統][産駒] テスコボーイ系,カノン 2010 鹿毛 [血統][産駒] FNo.[9-b],サクラバクシンオー 1989 鹿毛 [血統][産駒],ロージーミスト 1997 黒鹿毛 [血統][産駒],フォーティナイナーズサン Fourty Niners Son(米) 2001 栗毛 [血統]...,ピアノレッスン 1992 鹿毛 [血統][産駒],サクラユタカオー 1982 栗毛 [血統][産駒],サクラハゴロモ 1984 鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ビューティフルベーシック Beautiful Basic(米) 1989 栗毛 [血統][産駒],...,Conquistador Cielo,Rokeby Venus,パーソロン,スヰート,スノッブ,Chanel,リマンド,サンオーオク,グスタフ,アトランタ
2016101553,ルーラーシップ 2007 鹿毛 [血統][産駒] Mr. Prospector系,ミステリアスガール 2008 栗毛 [血統][産駒] FNo.[16-g],キングカメハメハ 2001 鹿毛 [血統][産駒],エアグルーヴ 1993 鹿毛 [血統][産駒],アグネスタキオン 1998 栗毛 [血統][産駒],エアイゾルテ 1997 栗毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],トニービン Tony Bin(愛) 1983 鹿毛 [血統][産駒],ダイナカール 1980 鹿毛 [血統][産駒],...,リマンド,イコマエイカン,Nearctic,Natalma,Victoria Park,Lady Angela,Grey Dawn,Relifordie,Hawaii,Tatallah
2018103120,ロードカナロア 2008 鹿毛 [血統][産駒] Mr. Prospector系,カレンシェリーメイ 2009 鹿毛 [血統][産駒] FNo.[13-c],キングカメハメハ 2001 鹿毛 [血統][産駒],レディブラッサム 1996 鹿毛 [血統][産駒],ディープインパクト 2002 鹿毛 [血統][産駒],スプリングチケット 1997 黒鹿毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],Storm Cat (米) 1983 黒鹿毛 [血統][産駒],サラトガデュー Saratoga Dew(米) 1989 鹿毛 [血統][産駒],...,Busted,Highclere,Kalamoun,State Pension,Hornbeam,Priddy Fair,Nijinsky,シル,ヴェンチア,シヤダイウイング
2017103309,Dark Angel 2005 芦毛 [血統][産駒] Northern Dancer系,ビーコンターン Beacon Tarn(英) 2010 鹿毛 [血統][産駒] FNo.[6-e],Acclamation 1999 鹿毛 [血統][産駒],Midnight Angel 1994 芦毛 [血統][産駒],Shamardal 2002 鹿毛 [血統][産駒],Baize 1993 栗毛

In [18]:
path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'

In [19]:
results.to_csv(path_mac+'results2.csv')
horse_results.to_csv(path_mac+'horse_results2.csv')
peds.to_csv(path_mac+'peds2.csv')
returns.to_csv(path_mac+'return2.csv')

In [20]:
add_return(returns)

2022-05-08 23:06:06,857 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-05-08 23:06:06,858 INFO sqlalchemy.engine.base.Engine ()
2022-05-08 23:06:06,860 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-05-08 23:06:06,861 INFO sqlalchemy.engine.base.Engine ()
2022-05-08 23:06:06,863 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-05-08 23:06:06,865 INFO sqlalchemy.engine.base.Engine SELECT return.id AS return_id, return.race_id AS return_race_id, return.kaime AS return_kaime, return.atari AS return_atari, return.haraimodoshi AS return_haraimodoshi, return.ninki AS return_ninki 
FROM return
2022-05-08 23:06:06,866 INFO sqlalchemy.engine.base.Engine ()
2022-05-08 23:06:09,294 INFO sqlalchemy.engine.base.Engine SELECT return.id AS return_id, return.race_id AS return_race_id, return.kaime AS return_kaime, return.atari AS return_atari, return.haraimodoshi AS return_haraimodoshi, re

In [21]:
add_horse_results(horse_results)

2022-05-08 23:37:12,863 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-05-08 23:37:12,866 INFO sqlalchemy.engine.base.Engine SELECT horse_results.id AS horse_results_id, horse_results.horse_id AS horse_results_horse_id, horse_results.date AS horse_results_date, horse_results.place AS horse_results_place, horse_results.weather AS horse_results_weather, horse_results.round_ AS horse_results_round_, horse_results.race_name AS horse_results_race_name, horse_results.eizo AS horse_results_eizo, horse_results.horse_len AS horse_results_horse_len, horse_results.wakuban AS horse_results_wakuban, horse_results.umaban AS horse_results_umaban, horse_results.odds AS horse_results_odds, horse_results.ninki AS horse_results_ninki, horse_results.chakujun AS horse_results_chakujun, horse_results.jockey AS horse_results_jockey, horse_results.kinryo AS horse_results_kinryo, horse_results.kyori AS horse_results_kyori, horse_results.baba AS horse_results_baba, horse_results.baba_index AS horse_re

KeyboardInterrupt: 

peds 正規化してない

In [ ]:
add_peds(peds)

In [ ]:
add_results(results)

In [ ]:
# db_session.rollback()

# CRUD操作

# 使用例 1

In [ ]:
db = db_session.query(Horse).all()


In [ ]:
len(db)

In [ ]:
for row in db:
    print(row.id,row.race_id,row.kaime,row.atari,row.haraimodoshi,row.ninki)
    break

# 使用例 2

In [ ]:
db = db_session.query(Horse.horse_id,Horse.chakujun).all()

In [ ]:
for row in db:
    print(row)
    break

# 使用例 3
select　操作

In [ ]:
db = db_session.query(Horse).filter(Horse.chakujun==1.0)

In [ ]:
for row in db:
    print(row.id,row.horse_id,row.peds_0,row.peds_61)
    break

 # 使用例 4
なんこデータを取ってくるか制限

In [ ]:
db = db_session.query(Horse).limit(20).all()

#  使用例5
昇順降順

In [ ]:
db = db_session.query(Horse).order_by(desc(Horse.chakujun)).limit(20)

In [ ]:
for row in db:
    print(row.chakujun)
    

# なんかrestart とかdata.dbコピーとかしまくったら治った

In [ ]:
db = db_session.query(Horse).filter(Horse.chakujun==1).first()

# Update 

In [ ]:
# db.horse_id = 1022
# db_session.commit()

# これで, 更新完了

# Delete

In [ ]:
# db = db_session.query(Horse).fillter(Horse.chakujunn==hoge).delete()

#  peds
重複チェック　と 追加だけ

In [ ]:
return_tables.head()

# return_tables
重複チェック, 追加

# horse_results
重複チェック, 追加

# results
重複チェック, 追加
race_id, 馬番で一意に定まる

# 更新

In [ ]:

# filtered_old = old[~old.index.isin(new.index)]

path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'
horse_results_new = pd.read_csv(path_mac+'horse_results_new.csv', index_col=0)
return_tables_new = pd.read_csv(path_mac+'returns_new.csv', index_col=0)
peds_new = pd.read_csv(path_mac+'peds_new.csv', index_col=0)
results_new = pd.read_csv(path_mac+'results_new.csv', index_col=0)

In [ ]:
peds_new[~peds.index.isin(peds_new.index)]

In [ ]:
peds_new